In [1]:
import mysql.connector

In [2]:
def convertLogToJson(result):
    try:
        send_data_arr = []
        resultList = list(result)
        TIERTYPE = ["St1.Nano","Mt2.GP","Lt3.Large","Lt4.Xlarge","Mt5.Prem"]
        RAM = [0.5,1.0,2.0,4.0,8.0]
        VCPUS = [2,2,2,2,2]
        NETWORKBANDWIDTH = [5,5,5,5,5]
        for value in resultList:
            send_data = {}
            send_data['tierType'] = TIERTYPE[value[1]]
            send_data['NetworkBandwidth'] = NETWORKBANDWIDTH[value[1]]
            send_data['RAM'] = RAM[value[1]]
            send_data['vCPUs'] = VCPUS[value[1]]
            send_data['IP_address']  = value[2]
            send_data_arr.append(send_data)
        return send_data_arr
    except KeyError:
        print('Execution Time: convert Log To Json --> KeyError')

In [3]:
def addDataToTheDataLog(mydb, tierType, IP_address):
    sql = "INSERT INTO data_log (tierType, IP_address) VALUES (%s, %s)"
    val = (tierType, str(IP_address))
    mycursor.execute(sql, val)
    mydb.commit()
    return True

In [4]:
mydb = mysql.connector.connect(
    host="database-1.ckbdn0ey3gu5.us-east-2.rds.amazonaws.com",
    user="admin",
    password="admin123",
    database="sdnDB"
)
mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM data_log")
myresult = mycursor.fetchall()

print(convertLogToJson(myresult))

[{'tierType': 'Mt2.GP', 'NetworkBandwidth': 5, 'RAM': 1.0, 'vCPUs': 2, 'IP_address': ' http://3.141.199.143'}, {'tierType': 'Lt3.Large', 'NetworkBandwidth': 5, 'RAM': 2.0, 'vCPUs': 2, 'IP_address': ' http://3.128.34.108'}, {'tierType': 'Lt4.Xlarge', 'NetworkBandwidth': 5, 'RAM': 4.0, 'vCPUs': 2, 'IP_address': ' http://18.216.198.194'}]


In [5]:
data = convertLogToJson(myresult)

In [6]:
import random
import requests
from server import Server

def sendData(data):
    try:
        #instantiating some servers
        s1 = Server(data[0]['vCPUs'],data[0]['RAM'],data[0]['IP_address']) #the total is 50
        s2 = Server(data[1]['vCPUs'],data[1]['RAM'],data[1]['IP_address']) #the total is 30
        s3 = Server(data[2]['vCPUs'],data[2]['RAM'],data[2]['IP_address']) #the total is 20

        total = (s1.RAM + s1.cpuCores) + (s2.RAM + s2.cpuCores) + (s3.RAM + s3.cpuCores)

        #calculating server selection probability for each server
        s1_prob = int(((s1.RAM + s1.cpuCores) / (total))*10)
        s2_prob = int(((s2.RAM + s2.cpuCores) / (total))*10)
        s3_prob = int(((s3.RAM + s3.cpuCores) / (total))*10)

        #server selection process
        for i in range(1,10):
            pos = {'server 1': s1_prob, 'server 2': s2_prob, 'server 3': s3_prob}
            choice = random.choice([x for x in pos for y in range(pos[x])]) #making the random choice

            if(choice == "server 1"):
                r =requests.get(s1.ip)
                print("IP address: ",s1.ip) 
                return s1.ip
            elif(choice == "server 2"):
                r =requests.get(s2.ip)
                print("IP address: ",s2.ip)
                return s2.ip
            else:
                r =requests.get(s3.ip)
                print("IP address: ",s3.ip)
                return s3.ip
        
    except KeyError:
        return False
    

In [7]:
sendData(convertLogToJson(myresult))

IP address:   http://18.216.198.194


' http://18.216.198.194'